# 0. Libraries

In [2]:
import os
from scipy.io.wavfile import read, write
import random
import numpy as np
import os
import wave
import pylab
from PIL import Image
import librosa
import shutil
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D, Lambda, Layer
import tensorflow as tf

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D, Lambda, Layer
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

In [4]:
import soundfile 

# 1. Create spectogram from wav files

In [4]:
def flac_to_wav(flac_path, wav_path):
    audio, sr = soundfile.read(flac_path)
    soundfile.write(wav_path, audio, sr, 'PCM_16')

In [5]:
def cutting_silence(path):
    rate, data = read(path)
    new_audio = librosa.effects.trim(np.array(read(path)[1],dtype=float), top_db = 20)[0]
    new_audio = new_audio[0:80000]
    #new_audio_2 = new_audio[-80000:]
    return (new_audio, rate)

In [6]:
def graph_spectrogram(sound, rate, sp_path):
    sound_info, frame_rate = sound, rate
    pylab.figure(num=None, figsize=(10,10))
    pylab.axis('off')
    pylab.specgram(sound_info, Fs=frame_rate)

    pylab.savefig(sp_path,pad_inches = 0, transparent = True, bbox_inches='tight')

In [19]:
for i in range (1, 151):
    os.mkdir("/Users/nastyastrashnova/Study/masters/BigDataset/audio/%s" %i)

In [20]:
for i in range(1,151):
    for audio in os.listdir("/Users/nastyastrashnova/Downloads/zw4p4p7sdh-1/differentPhrase/%s" % i):
        flac_to_wav("/Users/nastyastrashnova/Downloads/zw4p4p7sdh-1/differentPhrase/%s/%s" %(i, audio),
                "/Users/nastyastrashnova/Study/masters/BigDataset/audio/%s/%s.wav" %(i, audio[-6:-5]))

# add spectrograms

In [21]:
path_to_audio = "/Users/nastyastrashnova/Study/masters/BigDataset/audio/"
path_to_anchor = "/Users/nastyastrashnova/Study/masters/BigDataset/anchor/"
path_to_positive = "/Users/nastyastrashnova/Study/masters/BigDataset/positive/"
path_to_negative = "/Users/nastyastrashnova/Study/masters/BigDataset/negative/"

In [ ]:
num = 0
count = 200
for i in range(75):
    num = i*2 + 1
    path = (path_to_audio+"%s/" %num)
    # anchor
    sound, rate = cutting_silence(path + "1.wav")
    graph_spectrogram(sound, rate, path_to_anchor + "%s.png" %count)
    #print(path + "1.wav", "--------",  path_to_anchor + "%s.png" %count )

    sound, rate = cutting_silence(path + "2.wav")
    graph_spectrogram(sound, rate, path_to_anchor + "%s.png" %(count+1))
    #print(path + "2.wav", "--------",  path_to_anchor + "%s.png" %(count+1) )
               
                      
    # positive
    sound, rate = cutting_silence(path + "3.wav")
    graph_spectrogram(sound, rate, path_to_positive + "%s.png" %count)
    #print(path + "3.wav", "--------",  path_to_positive  + "%s.png" %(count) )
                      
    sound, rate = cutting_silence(path + "4.wav")
    graph_spectrogram(sound, rate, path_to_positive + "%s.png" %(count+1))
    #print(path + "4.wav", "--------",  path_to_positive  + "%s.png" %(count+1) )
                      
                      
    # negative
    num = i*2 + 2
    path = (path_to_audio+"%s/" %num)
                      
    sound, rate = cutting_silence(path + "1.wav")
    graph_spectrogram(sound, rate, path_to_negative + "%s.png" %count)
    #print(path + "1.wav", "--------",  path_to_negative  + "%s.png" %(count) )
                      
    sound, rate = cutting_silence(path + "2.wav")
    graph_spectrogram(sound, rate, path_to_negative + "%s.png" %(count+1))
    #print(path + "2.wav", "--------",  path_to_negative  + "%s.png" %(count+1) )
                      
    count = count + 2

                      

## Path to datasets

In [29]:
dataset_path = ('/Users/nastyastrashnova/Study/masters/BigDataset/')
ANC_PATH = (dataset_path + 'anchor')
POS_PATH = (dataset_path + 'positive')
NEG_PATH = (dataset_path + 'negative')

# 2. Generate Dataset

## 2.1 Prepare images 

In [31]:
anchor = tf.data.Dataset.list_files(ANC_PATH+'/*.png', shuffle=False)
positive = tf.data.Dataset.list_files(POS_PATH+'/*.png', shuffle=False)
negative = tf.data.Dataset.list_files(NEG_PATH+'/*.png', shuffle=False)

In [32]:
def preprocess(file_path):
    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)
    
    img = tf.image.resize(img, (100,100))

    img = img / 255.0

    return img

## 2.2 Pair images

In [33]:
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

In [34]:

def preprocess_twin(input_img, validation_img, label):
    return(preprocess(input_img), preprocess(validation_img), label)

In [35]:
# Build dataloader pipeline
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size=10000)

# Training partition
train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

# Testing partition
test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

# 3. Model

## 3.1 Embedding layer

In [5]:
def CNN(): 
    inp = Input(shape=(100,100,4), name='input_image')
    
    # First block
    c1 = Conv2D(64, (10,10), activation='relu')(inp)
    m1 = MaxPooling2D(64, (2,2), padding='same')(c1)
    
    # Second block
    c2 = Conv2D(128, (7,7), activation='relu')(m1)
    m2 = MaxPooling2D(64, (2,2), padding='same')(c2)
    
    # Third block 
    c3 = Conv2D(128, (4,4), activation='relu')(m2)
    m3 = MaxPooling2D(64, (2,2), padding='same')(c3)
    
    # Forth block 
    c3 = Conv2D(256, (4,4), activation='relu')(m2)
    m3 = MaxPooling2D(64, (2,2), padding='same')(c3)
    
    # Final embedding block
    c4 = Conv2D(256, (4,4), activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f1)
    
    
    return Model(inputs=[inp], outputs=[d1], name='embedding')

In [6]:
embedding = CNN()

## 3.2 Distance layer

In [8]:
# Siamese L1 Distance class
class LossFunction(Layer):
    
    # Init method - inheritance
    def __init__(self, **kwargs):
        super().__init__()
       
    # Magic happens here - similarity calculation
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [9]:
l1 = LossFunction()

In [10]:
def siamese_model(): 
    
    # Anchor image input in the network
    input_image = Input(name='input_img', shape=(100,100,4))
    
    # Validation image in the network 
    validation_image = Input(name='validation_img', shape=(100,100,4))
    
    # Combine siamese distance components
    siamese_layer = LossFunction()
    siamese_layer._name = 'distance'
    distances = siamese_layer(embedding(input_image), embedding(validation_image))
    
    # Classification layer 
    classifier = Dense(1, activation='sigmoid')(distances)
    
    return Model(inputs=[input_image, validation_image], outputs=classifier, name='SiameseNetwork')

In [11]:
siamese_model = siamese_model()

In [12]:
siamese_model.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_img (InputLayer)          [(None, 100, 100, 4) 0                                            
__________________________________________________________________________________________________
validation_img (InputLayer)     [(None, 100, 100, 4) 0                                            
__________________________________________________________________________________________________
embedding (Functional)          (None, 4096)         39753408    input_img[0][0]                  
                                                                 validation_img[0][0]             
__________________________________________________________________________________________________
distance (LossFunction)         (None, 4096)         0           embedding[0][0]     

# 4. Training 

## 4.1 Setup Loss and Optimizer

In [45]:
binary_cross_loss = tf.losses.BinaryCrossentropy()
opt = tf.keras.optimizers.Adam(1e-4) # 0.0001

## 4.2 Establish Checkpoints

In [46]:
checkpoint_dir = './training_checkpoints_2'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt=opt, siamese_model=siamese_model)

## 4.3 Build Train Step Function

In [47]:
def train_step(batch):
    
    # Record all of our operations 
    with tf.GradientTape() as tape:     
        # Get anchor and positive/negative image
        X = batch[:2]
        # Get label
        y = batch[2]
        
        # Forward pass
        yhat = siamese_model(X, training=True)
        # Calculate loss
        loss = binary_cross_loss(y, yhat)
    print(loss)
        
    # Calculate gradients
    grad = tape.gradient(loss, siamese_model.trainable_variables)
    
    # Calculate updated weights and apply to siamese model
    opt.apply_gradients(zip(grad, siamese_model.trainable_variables))
        
    # Return loss
    return loss

## 4.4 Build Training Loop

In [48]:
# Import metric calculations
from tensorflow.keras.metrics import Precision, Recall

In [49]:
def train(data, EPOCHS):
    # Loop through epochs
    for epoch in range(1, EPOCHS+1):
        print('\n Epoch {}/{}'.format(epoch, EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))
        
        # Creating a metric object 
        r = Recall()
        p = Precision()
        
        # Loop through each batch
        for idx, batch in enumerate(data):
            # Run train step here
            loss = train_step(batch)
            yhat = siamese_model.predict(batch[:2])
            r.update_state(batch[2], yhat)
            p.update_state(batch[2], yhat) 
            progbar.update(idx+1)
        print(loss.numpy(), r.result().numpy(), p.result().numpy())
        
        # Save checkpoints
        if epoch % 10 == 0: 
            checkpoint.save(file_prefix=checkpoint_prefix)

## 4.5 Train

In [ ]:
train(train_data, EPOCHS = 34)

In [56]:
train(train_data, EPOCHS = 3)


 Epoch 1/3
tf.Tensor(0.19025558, shape=(), dtype=float32)
29/29 [==============================] - 281s 10s/step
0.4248146 0.93721974 0.9675926

 Epoch 2/3
tf.Tensor(0.16472383, shape=(), dtype=float32)
29/29 [==============================] - 280s 10s/step
0.33026814 0.97510374 0.9710744

 Epoch 3/3
tf.Tensor(0.28520617, shape=(), dtype=float32)
25/29 [========================>.....] - ETA: 38stf.Tensor(0.19834813, shape=(), dtype=float32)


29/29 [==============================] - 281s 10s/step
0.071048774 0.9529914 0.9612069


In [57]:
r = Recall()
p = Precision()

for test_input, test_val, y_true in test_data.as_numpy_iterator():
    yhat = siamese_model.predict([test_input, test_val])
    r.update_state(y_true, yhat)
    p.update_state(y_true,yhat) 

print(r.result().numpy(), p.result().numpy())

0.9880952 0.90217394


# METRICS

In [190]:
def PresRecall(model, data, class_threshold=0.5):
    r = Recall()
    p = Precision()

    for test_input, test_val, y_true in data.as_numpy_iterator():
        yhat = model.predict([test_input, test_val])
        yhat = (np.concatenate(yhat) > class_threshold).astype(np.float32)
        
        r.update_state(y_true, yhat)
        p.update_state(y_true,yhat) 

    print("recall:",r.result().numpy(), "Precision",p.result().numpy())

In [194]:
def Accuracy(model, data, class_threshold=0.5):
    label = []
    predic = []
    for test_input, test_val, y_true in data.as_numpy_iterator():
        predic.append(model.predict([test_input, test_val]))
        label.append(y_true)

    label = np.concatenate( label, axis=0).tolist()
    prediction = np.concatenate( predic, axis=0).tolist()
    prediction = (np.concatenate(prediction) > class_threshold).astype(np.float32)

    print(accuracy_score(label, prediction))

In [198]:
PresRecall(siamese_model, test_data, 0.55)

recall: 0.99038464 Precision 0.94495416


In [195]:
Accuracy(siamese_model, test_data, 0.55)

0.9595959595959596


In [227]:
label = []
predic = []

In [228]:
label = []
predic = []
for test_input, test_val, y_true in test_data.as_numpy_iterator():
    predic.append(siamese_model.predict([test_input, test_val]))
    label.append(y_true)

In [211]:
#import numpy as np
import sklearn.metrics

"""
Python compute equal error rate (eer)
ONLY tested on binary classification

:param label: ground-truth label, should be a 1-d list or np.array, each element represents the ground-truth label of one sample
:param pred: model prediction, should be a 1-d list or np.array, each element represents the model prediction of one sample
:param positive_label: the class that is viewed as positive class when computing EER
:return: equal error rate (EER)
"""
def compute_eer(label, pred, positive_label=1):
    # all fpr, tpr, fnr, fnr, threshold are lists (in the format of np.array)
    fpr, tpr, threshold = sklearn.metrics.roc_curve(label, pred, positive_label)
    fnr = 1 - tpr

    # the threshold of fnr == fpr
    eer_threshold = threshold[np.nanargmin(np.absolute((fnr - fpr)))]

    # theoretically eer from fpr and eer from fnr should be identical but they can be slightly differ in reality
    eer_1 = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
    eer_2 = fnr[np.nanargmin(np.absolute((fnr - fpr)))]

    # return the mean of eer from fpr and from fnr
    eer = (eer_1 + eer_2) / 2
    return eer

In [226]:
#rom compute_eer import compute_eer
label = np.concatenate( label, axis=0).tolist()
prediction = np.concatenate( predic, axis=0).tolist()
prediction = (np.concatenate(prediction) > 0.55).astype(np.float32)
eer = compute_eer(label, prediction)
print('The equal error rate is {:.3f}'.format(eer))

The equal error rate is 0.375


In [231]:
0.045*100

4.5

In [229]:
label = np.concatenate( label, axis=0).tolist()
prediction = np.concatenate( predic, axis=0).tolist()
prediction = (np.concatenate(prediction) > 0.55).astype(np.float32)
eer = compute_eer(label, prediction)
print('The equal error rate is {:.3f}'.format(eer))

The equal error rate is 0.045
